# 04 — Batch Results + Per-Class Analysis (Primary DP)

This notebook runs the **primary DP/beam decoder** over the 29-item test set,
optionally performs **EN TTS → HU ASR**, and computes **WER / CER / PER**.


In [3]:

from pathlib import Path
import pandas as pd

DATA = Path("./data")
AUDIO_DIR = DATA / "audio_run"
AUDIO_DIR.mkdir(exist_ok=True, parents=True)
RESULTS_CSV = DATA / "results/results_morph_ngram_dp_v2.csv"
OUT = DATA / "results/results_morph_ngram_dp_scored_v2.csv"

#import matplotlib
#matplotlib.use("Qt5Agg")

import matplotlib.pyplot as plt
%matplotlib inline

print(RESULTS_CSV)

data/results/results_morph_ngram_dp_v2.csv


## Per-class analysis (tags for Appendix A)

In [4]:
# Rough tags per item index
ITEM_TAGS = {
 1: ["front_rounded","sibilants","affricate_cs","clusters","vowel_length"],                 # Mit sütsz kis szűcs
 2: ["front_rounded","vowel_length","plosives","rhythm"],                                    # Sárga bögre görbe bögre
 3: ["plosives","front_rounded","vowel_length","rhythm"],                                    # Fekete bikapata...
 4: ["vowel_length","sibilants","plosives","rhythm"],                                        # Két pék két szép...
 5: ["hiatus","final_long_e","vowel_length"],                                                # Jamaika a jamaikaiaké
 6: ["sibilant_clusters","affricates","affricate_cs","long_i","vowel_length"],               # A szecsuáni síncsiszoló...
 7: ["palatal_ty","palatal_ny","geminate","vowel_length"],                                   # Jobb egy lúdnyak...
 8: ["vowel_length","laterals"],                                                             # Lali a lila ló elalél
 9: ["vowel_length","vowel_harmony"],                                                        # Nem lehet a Márta...
10: ["palatal_gy","front_rounded","vowel_length"],                                          # Gyűrűt újra fűz
11: ["palatal_ny","vowel_length"],                                                           # Hány nyár
12: ["palatal_ty","affricate_cs","vowel_length"],                                           # Tyúk és csibe
13: ["affricate_dzs","word_initial","front_rounded","vowel_length"],                        # Dzsinn ül a dzsámi előtt
14: ["s_vs_sh","vowel_length","sibilants"],                                                  # Sárban szárad (s vs sz)
15: ["z_vs_zh","vowel_length","sibilants"],                                                  # Zsákban a zsemle (zs vs z)
16: ["voicing_assimilation_target","plosives"],                                             # Egy nagy kertben (t+b → d)
17: ["front_rounded_long","front_rounded_short","vowel_length"],                            # Őr űzi az ürgét
18: ["front_rounded","vowel_length"],                                                       # Két bögre teát kér
19: ["long_o","front_rounded_long","affricate_cs","vowel_length"],                          # A kórus a kocsma előtt áll
20: ["front_rounded_short","front_rounded_long","sibilants","clusters","vowel_length"],     # Szép zöld fű
21: ["front_rounded_short","front_rounded_long","sibilants","vowel_length"],                # Sütőtökből sütit süt
22: ["i_vs_long_i","geminate_dd","vowel_length"],                                           # Hidd el itt írok
23: ["long_a","i_vs_long_i_rhyme","vowel_length"],                                          # Vár a víz
24: ["front_rounded_long","sibilants","vowel_length"],                                      # Füstöl a fűrész
25: ["palatal_gy","palatal_gy_clusters","long_u","vowel_length","clusters"],                # Gyúl a Gyertya
26: ["long_vs_short_vowels","vowel_length"],                                                # Ég a lámpa
27: ["affricate_cs_onset","front_rounded_long","voiced_coda_g","vowel_length"],             # Csőből csöpög
28: ["front_rounded_long","clusters","sibilants","vowel_length"],                           # Kő őrzi őzt
}

def explode_classes(df):
    recs = []
    for i, r in df.reset_index(drop=True).iterrows():
        idx = i + 1
        for tag in ITEM_TAGS.get(idx, []):
            rec = dict(r)
            rec["item"] = idx
            rec["class"] = tag
            recs.append(rec)
    return pd.DataFrame(recs)

def agg_metric(s):
    s = s.dropna()
    return None if s.empty else float(s.mean())

def mean_or_none(series):
    s = series.dropna()
    return None if s.empty else float(s.mean())


In [5]:
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)
dfc


,hu_text,en_text_morph,morph_cost,time,hu_hyp_morph,morph_wer,morph_cer,morph_per,item,class
0,Mit sütsz kis szűcs,meet shoot speaking,3.361667,5.422086,Mégsőt szépen!,1.0,0.736842,1.250000,1,front_rounded
1,Mit sütsz kis szűcs,meet shoot speaking,3.361667,5.422086,Mégsőt szépen!,1.0,0.736842,1.250000,1,sibilants
2,Mit sütsz kis szűcs,meet shoot speaking,3.361667,5.422086,Mégsőt szépen!,1.0,0.736842,1.250000,1,affricate_cs
3,Mit sütsz kis szűcs,meet shoot speaking,3.361667,5.422086,Mégsőt szépen!,1.0,0.736842,1.250000,1,clusters
4,Mit sütsz kis szűcs,meet shoot speaking,3.361667,5.422086,Mégsőt szépen!,1.0,0.736842,1.250000,1,vowel_length
...,...,...,...,...,...,...,...,...,...,...
90,Csőből csöpög,transformed,3.785000,2.907422,Transzformed.,1.0,1.000000,2.100000,27,vowel_length
91,Kő őrzi őzt,co is e east,4.040000,3.245720,Kóissi East!,1.0,0.818182,1.444444,28,front_rounded_long
92,Kő őrzi őzt,co is e east,4.040000,3.245720,Kóissi East!,1.0,0.818182,1.444444,28,clusters
93,Kő őrzi őzt,co is e east,4.040000,3.245720,Kóissi East!,1.0,0.818182,1.444444,28,sibilants


## Summaries

In [6]:
summary = dfc.groupby("class").agg({
    "morph_wer": agg_metric,
    "morph_cer": agg_metric,
    "morph_per": agg_metric,
    "morph_cost": "mean"
}).rename(columns={"morph_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/morph_dp_aggregation_v2.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary

,class,morph_wer,morph_cer,morph_per,avg_search_cost
0,affricate_cs,0.958333,0.780765,1.316585,6.911478
1,affricate_cs_onset,1.000000,1.000000,2.100000,3.785000
2,affricate_dzs,0.800000,0.916667,2.214286,6.241389
3,affricates,1.000000,0.770833,1.294118,12.942857
4,clusters,1.000000,0.850066,1.562500,3.643750
5,final_long_e,1.000000,0.809524,1.304348,8.420000
6,front_rounded,0.998413,0.821773,1.436189,7.261713
7,front_rounded_long,0.976190,0.813953,1.514286,4.702602
8,front_rounded_short,1.000000,0.722222,1.324074,4.392183
9,geminate,1.000000,0.812500,1.500000,8.019167


In [7]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"morph_{tag}_avg"] = mean_or_none(df[f"morph_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
morph_wer_avg,1.022279
morph_cer_avg,0.883285
morph_per_avg,1.494531


In [ ]:
RESULTS_CSV = Path("./data/results/results_morph_ngram_time.csv")
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "morph_wer": agg_metric,
    "morph_cer": agg_metric,
    "morph_per": agg_metric,
    "morph_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/morph_ngram_time_aggregation.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary

In [8]:
RESULTS_CSV = DATA / "results/results_morph_ngram_time.csv"
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "morph_wer": agg_metric,
    "morph_cer": agg_metric,
    "morph_per": agg_metric,
    "morph_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/morph_ngram_time_aggregation.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary

,class,morph_wer,morph_cer,morph_per,morph_cost
0,affricate_cs,1.000000,0.913440,1.597222,7.130004
1,affricate_cs_onset,1.000000,1.000000,2.100000,3.785000
2,affricate_dzs,1.000000,0.791667,1.500000,6.241389
3,affricates,1.000000,0.854167,1.500000,13.218056
4,clusters,1.000000,0.859635,1.611111,3.880083
5,final_long_e,1.000000,0.904762,1.521739,9.022778
6,front_rounded,1.000000,0.838307,1.457317,7.737046
7,front_rounded_long,1.047619,0.950200,1.764286,5.132850
8,front_rounded_short,1.111111,0.938889,1.712963,4.984905
9,geminate,1.000000,0.875000,1.545455,8.437917


In [9]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"morph_{tag}_avg"] = mean_or_none(df[f"morph_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
morph_wer_avg,1.000000
morph_cer_avg,0.841333
morph_per_avg,1.464960


In [ ]:
RESULTS_CSV = DATA / "results/results_dp_vs_gemini_8k.csv"
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "dp_wer": agg_metric,
    "dp_cer": agg_metric,
    "dp_per": agg_metric,
    "dp_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/dp_8k_v1_aggregation.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary


,class,dp_wer,dp_cer,dp_per,avg_search_cost
0,affricate_cs,1.000000,0.709240,1.054330,10.460625
1,affricate_cs_onset,1.000000,0.846154,1.200000,7.771500
2,affricate_dzs,1.000000,0.875000,1.714286,10.760000
3,affricates,1.000000,0.604167,0.911765,15.411000
4,clusters,1.000000,0.887432,1.451389,6.024125
5,final_long_e,1.333333,0.809524,1.086957,13.669500
6,front_rounded,1.398810,1.203139,1.241676,11.274750
7,front_rounded_long,0.964286,0.834366,1.290476,8.676000
8,front_rounded_short,0.916667,0.722222,1.314815,8.334333
9,geminate,1.400000,0.750000,1.318182,14.907000


In [27]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"dp_{tag}_avg"] = mean_or_none(df[f"dp_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
dp_wer_avg,1.164626
dp_cer_avg,0.894636
dp_per_avg,1.204324


In [ ]:
RESULTS_CSV = DATA / "results/results_dp_vs_gemini_4_5zipf.csv"
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "dp_wer": agg_metric,
    "dp_cer": agg_metric,
    "dp_per": agg_metric,
    "dp_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/dp_4_5zipf_v1_aggregation.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary

,class,dp_wer,dp_cer,dp_per,avg_search_cost
0,affricate_cs,1.083333,0.691612,0.984886,10.672000
1,affricate_cs_onset,1.500000,1.230769,2.100000,8.400000
2,affricate_dzs,1.000000,0.750000,1.214286,10.760000
3,affricates,1.333333,0.687500,0.911765,16.075000
4,clusters,1.083333,0.887432,1.451389,6.480500
5,final_long_e,1.666667,1.000000,1.217391,15.029000
6,front_rounded,1.190476,0.793140,1.293844,12.663000
7,front_rounded_long,1.285714,1.151215,1.680952,9.208143
8,front_rounded_short,1.500000,1.244444,1.916667,9.068000
9,geminate,1.400000,0.718750,1.318182,14.934000


In [23]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"dp_{tag}_avg"] = mean_or_none(df[f"dp_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
dp_wer_avg,1.150340
dp_cer_avg,0.886090
dp_per_avg,1.275127


In [24]:
RESULTS_CSV = DATA / "results/results_dp_vs_gemini_5zipf.csv"
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "dp_wer": agg_metric,
    "dp_cer": agg_metric,
    "dp_per": agg_metric,
    "dp_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/dp_5zipf_v1_aggregation.csv"
df.to_csv(AGGREGATION_CSV, index=False)
summary

,class,dp_wer,dp_cer,dp_per,avg_search_cost
0,affricate_cs,1.083333,0.717991,0.990196,9.395250
1,affricate_cs_onset,1.000000,0.769231,1.200000,7.823000
2,affricate_dzs,1.000000,0.791667,1.071429,10.760000
3,affricates,1.333333,0.625000,0.794118,12.976000
4,clusters,1.000000,0.915470,1.319444,5.705250
5,final_long_e,7.333333,5.142857,4.000000,13.819000
6,front_rounded,1.353175,0.946412,1.293273,10.756333
7,front_rounded_long,1.011905,0.862204,1.238889,8.635429
8,front_rounded_short,1.027778,0.777778,1.222222,8.196667
9,geminate,1.400000,0.750000,1.318182,14.934000


In [25]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"dp_{tag}_avg"] = mean_or_none(df[f"dp_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
dp_wer_avg,1.380187
dp_cer_avg,1.154610
dp_per_avg,1.343032


In [11]:
# V2
DATA = Path("./data")

In [12]:
RESULTS_CSV = DATA / "results/results_dp_vs_gemini_4_5zipf.csv"
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "dp_wer": agg_metric,
    "dp_cer": agg_metric,
    "dp_per": agg_metric,
    "dp_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/dp_4_5zipf_v2_aggregation.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary

,class,dp_wer,dp_cer,dp_per,avg_search_cost
0,affricate_cs,3.062500,2.948212,3.794118,9.278500
1,affricate_cs_onset,1.000000,0.846154,1.300000,8.000000
2,affricate_dzs,1.000000,0.750000,0.714286,7.800000
3,affricates,2.166667,0.833333,1.176471,14.082000
4,clusters,2.812500,3.407482,4.444444,5.651500
5,final_long_e,2.000000,1.190476,0.913043,5.556000
6,front_rounded,2.315476,2.313723,2.873926,9.652333
7,front_rounded_long,1.166667,1.019347,1.372222,7.510286
8,front_rounded_short,1.444444,1.166667,1.500000,7.035000
9,geminate,1.400000,0.750000,1.363636,12.611000


In [13]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"dp_{tag}_avg"] = mean_or_none(df[f"dp_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
dp_wer_avg,1.440221
dp_cer_avg,1.180830
dp_per_avg,1.550016


In [13]:
RESULTS_CSV = DATA / "results/results_dp_vs_gemini_5zipf.csv"
df = pd.read_csv(RESULTS_CSV)
dfc = explode_classes(df)

summary = dfc.groupby("class").agg({
    "dp_wer": agg_metric,
    "dp_cer": agg_metric,
    "dp_per": agg_metric,
    "dp_cost": "mean"
}).rename(columns={"dp_cost":"avg_search_cost"}).reset_index().sort_values("class")

AGGREGATION_CSV = DATA / "aggregation/dp_5zipf_v2_aggregation.csv"
summary.to_csv(AGGREGATION_CSV, index=False)
summary

,class,dp_wer,dp_cer,dp_per,avg_search_cost
0,affricate_cs,1.083333,0.717991,0.990196,9.395250
1,affricate_cs_onset,1.000000,0.769231,1.200000,7.823000
2,affricate_dzs,1.000000,0.791667,1.071429,10.760000
3,affricates,1.333333,0.625000,0.794118,12.976000
4,clusters,1.000000,0.915470,1.319444,5.705250
5,final_long_e,7.333333,5.142857,4.000000,13.819000
6,front_rounded,1.353175,0.946412,1.293273,10.756333
7,front_rounded_long,1.011905,0.862204,1.238889,8.635429
8,front_rounded_short,1.027778,0.777778,1.222222,8.196667
9,geminate,1.400000,0.750000,1.318182,14.934000


In [12]:
summary = {
    "n_items": len(df),
}

for tag in ["wer","cer","per"]:
    summary[f"dp_{tag}_avg"] = mean_or_none(df[f"dp_{tag}"])

pd.DataFrame([summary]).T

,0
n_items,28.000000
dp_wer_avg,1.420323
dp_cer_avg,1.319986
dp_per_avg,1.574185
